In [1]:
import pandas as pd
pd.set_option('display.max_rows', 5)

In [2]:
df = pd.read_csv('./data-temp/daily.csv', encoding='utf-8')

In [3]:
df.drop(df.columns[[0]], axis=1, inplace=True)
df.drop(['U', 'V', 'temp', 'rh', 'psfc'], axis='columns', inplace=True)
df['date'] = pd.to_datetime(df['date'])

In [4]:
P_sum = df[['pm2.5', 'pm10', 'so2', 'no2', 'co', 'o3']].sum(axis=1)
# 计算每个城市在所有日期的污染物百分比成分谱（归一化）
# 以pm2.5为例
df['P_pm25'] = df['pm2.5'].iloc[:] / P_sum

In [5]:
df['year'] = df['date'].dt.year  # 提取年份
df['month'] = df['date'].dt.month  # 提取月份
df['day'] = df['date'].dt.day  # 提取天数

In [6]:
city = df['city']
city = city.drop_duplicates()
city = list(city)

In [7]:
month_31 = [1, 3, 5, 7, 8, 10, 12]  # 31天的月份

In [ ]:
# 计算某市这个月的pm2.5百分比成分谱`均值`
P_pm25_ = list()
sum_ = 0  # 计数器
for i in range(len(city)):
    for j in range(len(df['city'])):
        if df['city'][j] == city[i]:  # 选取某市的数据
            if df['year'][j] == 2016:  # 闰年
                if df['month'][j] in month_31:  # 31天的月份
                    sum_ += df['P_pm25'][j]
                    # 算完一个月，计数器清零
                    if df['day'][j] == 31:
                        P_pm25_.append([df['city'][j], df['year'][j], df['month'][j], sum_ / 31])  # 均值
                        sum_ = 0
                elif df['month'][j] == 2:  # 闰年2月
                    sum_ += df['P_pm25'][j]
                    if df['day'][j] == 29:
                        P_pm25_.append([df['city'][j], df['year'][j], df['month'][j], sum_ / 29])
                        sum_ = 0
                else:  # 30天的月份
                    sum_ += df['P_pm25'][j]
                    if df['day'][j] == 30:
                        P_pm25_.append([df['city'][j], df['year'][j], df['month'][j], sum_ / 30])
                        sum_ = 0
            else:  # 非闰年
                if df['month'][j] in month_31:  # 31天的月份
                    sum_ += df['P_pm25'][j]
                    if df['day'][j] == 31:
                        P_pm25_.append([df['city'][j], df['year'][j], df['month'][j], sum_ / 31])
                        sum_ = 0
                elif df['month'][j] == 2:  # 非闰年2月
                    sum_ += df['P_pm25'][j]
                    if df['day'][j] == 28:
                        P_pm25_.append([df['city'][j], df['year'][j], df['month'][j], sum_ / 28])
                        sum_ = 0
                else:  # 30天的月份
                    sum_ += df['P_pm25'][j]
                    if df['day'][j] == 30:
                        P_pm25_.append([df['city'][j], df['year'][j], df['month'][j], sum_ / 30])
                        sum_ = 0
P_pm25_ = pd.DataFrame(P_pm25_)
P_pm25_.columns = ['city', 'year', 'month', 'P_pm25_']
P_pm25_.to_csv('P_pm25_.csv', encoding='utf-8-sig', index=False)  # 保存一下中间结果：均值

In [ ]:
P_pm25_ = pd.read_csv('./P_pm25_.csv', encoding='utf-8')

In [ ]:
# 计算某市，某日，某污染物（如pm2.5）的当日特征值
# C_pm25 = P_pm25 / P_pm25_
C_pm25 = list()
for i in range(len(city)):
    for j in range(len(df['city'])):
        if df['city'][j] == city[i]:  # 选取某市数据
            for k in range(len(P_pm25_['city'])):
                if df['city'][j] == P_pm25_['city'][k] and \
                        df['year'][j] == P_pm25_['year'][k] and \
                        df['month'][j] == P_pm25_['month'][k]:
                    C_pm25.append([df['city'][j], df['year'][j],
                                   df['month'][j], df['day'][j],
                                   df['P_pm25'][j] / P_pm25_['P_pm25_'][k]])
C_pm25 = pd.DataFrame(C_pm25)
C_pm25.columns = ['city', 'year', 'month', 'day', 'C_pm25_']

In [ ]:
# 保存数据
C_pm25.to_csv('C_pm25.csv', encoding='utf-8-sig', index=False)

In [8]:
# 读取数据
C_pm25 = pd.read_csv('./C_pm25.csv', encoding='utf-8')

In [9]:
df

,province,city,pm2.5,pm10,so2,no2,co,o3,date,P_pm25,year,month,day
0,上海市,上海市,73.768400,105.406800,42.533600,69.246400,1.592000,16.167200,2013-01-01,0.238954,2013,1,1
1,云南省,临沧市,25.521759,27.578333,8.968889,2.099907,0.294167,62.829259,2013-01-01,0.200497,2013,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
808477,黑龙江省,黑河市,3.953824,5.476865,1.944138,3.111097,0.272727,51.312602,2018-12-31,0.059842,2018,12,31
808478,黑龙江省,齐齐哈尔市,22.555276,26.401759,6.897085,7.511457,0.422613,44.790000,2018-12-31,0.207733,2018,12,31


In [10]:
C_pm25

,city,year,month,day,C_pm25_
0,上海市,2013,1,1,0.869471
1,上海市,2013,1,2,0.791364
...,...,...,...,...,...
808477,齐齐哈尔市,2018,12,30,0.746409
808478,齐齐哈尔市,2018,12,31,1.167190


In [12]:
# 把C_pm25添加到df
df = pd.merge(df, C_pm25, on=['city', 'year', 'month', 'day'], how='inner')

In [13]:
df

,province,city,pm2.5,pm10,so2,no2,co,o3,date,P_pm25,year,month,day,C_pm25_
0,上海市,上海市,73.768400,105.406800,42.533600,69.246400,1.592000,16.167200,2013-01-01,0.238954,2013,1,1,0.869471
1,云南省,临沧市,25.521759,27.578333,8.968889,2.099907,0.294167,62.829259,2013-01-01,0.200497,2013,1,1,1.172797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
808477,黑龙江省,黑河市,3.953824,5.476865,1.944138,3.111097,0.272727,51.312602,2018-12-31,0.059842,2018,12,31,0.477289
808478,黑龙江省,齐齐哈尔市,22.555276,26.401759,6.897085,7.511457,0.422613,44.790000,2018-12-31,0.207733,2018,12,31,1.167190


In [14]:
# 删除中间数据：P_pm25, year, month, day
df.drop(['P_pm25', 'year', 'month', 'day'], axis=1, inplace=True)

In [16]:
df.drop(['pm2.5', 'pm10', 'so2', 'no2', 'co', 'o3'], axis=1, inplace=True)

In [17]:
df

,province,city,date,C_pm25_
0,上海市,上海市,2013-01-01,0.869471
1,云南省,临沧市,2013-01-01,1.172797
...,...,...,...,...
808477,黑龙江省,黑河市,2018-12-31,0.477289
808478,黑龙江省,齐齐哈尔市,2018-12-31,1.167190


In [18]:
# 保存数据
df.to_csv('EI_daily_pm25.csv', encoding='utf-8-sig', index=False)